In [183]:
import moritz.lindner.masterarbeit.epa.construction.builder.ExtendedPrefixAutomatonBuilder
import moritz.lindner.masterarbeit.epa.construction.builder.SampleEventMapper
import moritz.lindner.masterarbeit.epa.*
import moritz.lindner.masterarbeit.epa.domain.*
import java.io.File

val epa =
    ExtendedPrefixAutomatonBuilder<Long>()
        .setFile(File("/Users/moritzlindner/programming/Masterarbeit/epa-visualizer/epa/src/test/resources/simple.xes"))
        .setEventLogMapper(SampleEventMapper())
        .build()

epa

simple.xes
j,i,root,a,f,d,e,h,c,b,g
a,b,c,d,e,f,g,h,i,j
Transition(start=d, activity=g, end=g),Transition(start=e, activity=h, end=h),Transition(start=b, activity=d, end=d),Transition(start=c, activity=e, end=e),Transition(start=c, activity=f, end=f),Transition(start=root, activity=a, end=a),Transition(start=a, activity=b, end=b),Transition(start=a, activity=c, end=c),Transition(start=h, activity=i, end=i),Transition(start=h, activity=j, end=j)
j:4,i:2,root:0,a:1,f:3,d:1,e:2,h:2,c:2,b:1,g:1
j:Event(activity=j, timestamp=1745625840000, caseIdentifier=3, predecessor=Event(activity=h, timestamp=1745625780000, caseIdentifier=3, predecessor=null)),i:Event(activity=i, timestamp=1745625840000, caseIdentifier=2, predecessor=Event(activity=h, timestamp=1745625780000, caseIdentifier=2, predecessor=null)),a:Event(activity=a, timestamp=1745625600000, caseIdentifier=1, predecessor=null),Event(activity=a, timestamp=1745625600000, caseIdentifier=2, predecessor=null),Event(activity=a, timestamp=174562

In [184]:
val childrenByParent = epa.transitions.groupBy { it.start }.mapValues { it.value.map { it.end } }
childrenByParent

{d=[g], e=[h], b=[d], c=[e, f], root=[a], a=[b, c], h=[i, j]}

In [185]:
import com.google.common.collect.ArrayListMultimap
import com.google.common.collect.Multimap

fun <T> invertMap(map: Map<T, List<T>>): Multimap<T, T> {
    val multimap: Multimap<T, T> = ArrayListMultimap.create()
    for ((key, values) in map) {
        for (value in values) {
            multimap.put(value, key)
        }
    }
    return multimap
}
val parentsByState = invertMap(childrenByParent)
parentsByState

{j=[h], i=[h], a=[root], f=[c], d=[b], e=[c], h=[e], c=[a], g=[d], b=[a]}

In [186]:
val childrenByParentWithOneChildren = childrenByParent.filterValues { it.size == 1 }.filterKeys { it as? State.PrefixState != null }.mapValues { it.value.first() as State.PrefixState }
childrenByParentWithOneChildren

{d=g, e=h, b=d}

In [187]:
val activitiesToRemove = childrenByParentWithOneChildren.values.map { it.via }
activitiesToRemove

[g, h, d]

In [188]:
val preStates = epa.states.filter { !childrenByParentWithOneChildren.keys.contains(it) }
preStates

[j, i, root, a, f, h, c, g]

In [189]:
val prePartitionByState = epa.states.associateWith { epa.partition(it) }
prePartitionByState

{j=4, i=2, root=0, a=1, f=3, d=1, e=2, h=2, c=2, b=1, g=1}

In [190]:
val preActivities = epa.activities
preActivities

[a, b, c, d, e, f, g, h, i, j]

In [191]:
val preSeqByState = preStates.associateWith { epa.sequence(it) }
preSeqByState

{j=[Event(activity=j, timestamp=1745625840000, caseIdentifier=3, predecessor=Event(activity=h, timestamp=1745625780000, caseIdentifier=3, predecessor=null))], i=[Event(activity=i, timestamp=1745625840000, caseIdentifier=2, predecessor=Event(activity=h, timestamp=1745625780000, caseIdentifier=2, predecessor=null))], root=[], a=[Event(activity=a, timestamp=1745625600000, caseIdentifier=1, predecessor=null), Event(activity=a, timestamp=1745625600000, caseIdentifier=2, predecessor=null), Event(activity=a, timestamp=1745625600000, caseIdentifier=3, predecessor=null), Event(activity=a, timestamp=1745625600000, caseIdentifier=4, predecessor=null)], f=[Event(activity=f, timestamp=1745625720000, caseIdentifier=4, predecessor=Event(activity=c, timestamp=1745625660000, caseIdentifier=4, predecessor=null))], h=[Event(activity=h, timestamp=1745625780000, caseIdentifier=2, predecessor=Event(activity=e, timestamp=1745625720000, caseIdentifier=2, predecessor=null)), Event(activity=h, timestamp=1745625

In [113]:
fun followChain(a: State, mappings: Map<State, List<State>>, acc: List<State>): List<State> {
    if (mappings[a] != null) {
        val n = mappings[a]!!.first()
        return if (mappings[a]!!.size == 1) followChain(n, mappings, acc + n)
        else acc
    } else return acc
}

In [ ]:
fun mergeLists(lists: List<List<State>>): List<List<State>> {
    return lists.filter { current ->
        // Keep only those lists that are not strict subsets of any other
        lists.none { other ->
            current != other && other.containsAll(current)
        }
    }
}

In [192]:
// merging need happen because of no order garentee when iterating singleTargetStates
val umergedLists = childrenByParentWithOneChildren.map { singleTargetState ->
    listOf(singleTargetState.key) + followChain(singleTargetState.key, childrenByParent, emptyList())
}
val chains = mergeLists(umergedLists)
chains

[[e, h], [b, d, g]]

In [193]:
val activityByChain = chains.associateWith { chain ->
    chain.fold(Activity("")) { a, b ->
        Activity(a.name + (b as State.PrefixState).name)
    }
}
activityByChain

{[e, h]=eh, [b, d, g]=bdg}

In [194]:
val newActivities = preActivities + activityByChain.values
newActivities

[a, b, c, d, e, f, g, h, i, j, eh, bdg]

In [195]:
val syntheticStateByChain = chains.associateWith { chain ->
    chain.reduce { a, b ->
        State.PrefixState(
            from = (a as State.PrefixState).from,
            via = activityByChain[chain]!!
        )
    }
}
syntheticStateByChain

{[e, h]=eh, [b, d, g]=bdg}

In [196]:
val synthetics = syntheticStateByChain.values
synthetics

[eh, bdg]

In [206]:
val childrenBySynthetic = chains.associateWith { chain ->
    val last = chain.last()
    val targets = (childrenByParent[last] ?: emptyList())

    targets.map { target ->
        // if target is synthetic
        if (chains.flatten().contains(target)) {
            println("yes")
            val newFrom = syntheticByChainparts[target]!!
            println("newFrom for $target is $newFrom")
            newFrom
        } else State.PrefixState(
            from = syntheticStateByChain[chain]!!,
            via = (target as State.PrefixState).via
        )
    }
}.mapKeys { syntheticStateByChain[it.key]!! }

[[eh, eh], []]

In [127]:
import moritz.lindner.masterarbeit.epa.domain.Event

val seqByChain = chains.associateWith { chain ->
    chain.fold(emptyList<Event<Long>>()) { acc, state ->
        acc + epa.sequence(state)
    }
}
seqByChain.mapKeys { syntheticStateByChain[it.key] }

{fe=[Event(activity=f, timestamp=1745629440000, caseIdentifier=2, predecessor=Event(activity=d, timestamp=1745629380000, caseIdentifier=2, predecessor=null)), Event(activity=e, timestamp=1745629500000, caseIdentifier=2, predecessor=Event(activity=f, timestamp=1745629440000, caseIdentifier=2, predecessor=null))], bcd=[Event(activity=b, timestamp=1745625660000, caseIdentifier=1, predecessor=Event(activity=a, timestamp=1745625600000, caseIdentifier=1, predecessor=null)), Event(activity=b, timestamp=1745636460000, caseIdentifier=4, predecessor=Event(activity=a, timestamp=1745636400000, caseIdentifier=4, predecessor=null)), Event(activity=c, timestamp=1745625720000, caseIdentifier=1, predecessor=Event(activity=b, timestamp=1745625660000, caseIdentifier=1, predecessor=null)), Event(activity=c, timestamp=1745636520000, caseIdentifier=4, predecessor=Event(activity=b, timestamp=1745636460000, caseIdentifier=4, predecessor=null)), Event(activity=d, timestamp=1745625780000, caseIdentifier=1, pred

In [121]:
val parByChain = chains.associateWith { chain ->
    epa.partition(chain.first())
}
parByChain

{[f, e]=2, [b, c, d]=1, [c, b, d]=2, [f, e]=1}

In [129]:
val syntheticStateByChainStart = syntheticStateByChain.mapKeys { a -> a.key.first() }
syntheticStateByChainStart

{f=fe, b=bcd, c=cbd, f=fe}

In [170]:
val syntheticByParent = buildMap<State, List<State>> {
    childrenByParent.forEach { (parent, children) ->
        children.forEach { child ->
            if (syntheticStateByChainStart[child] != null) {
                val alreadyPresnt = get(parent) ?: emptyList<State>()
                put(parent, listOf(syntheticStateByChainStart[child]!!) + alreadyPresnt)
            }
        }
    }
}
syntheticByParent

{a=[bcd]}

In [ ]:
fun <T> mergeMaps(
    first: Map<T, List<T>>,
    second: Map<T, List<T>>
): Map<T, List<T>> {
    return (first.keys + second.keys).associateWith { key ->
        (first[key].orEmpty() + second[key].orEmpty())
    }
}

In [175]:
val mappingWithouthSynthStyle = buildMap<State, List<State>> {
    childrenByParent.forEach { parent, children ->
        if (childrenByParentWithOneChildren[parent] != null) return@forEach
        if (chains.flatten().contains(parent)) return@forEach
        val newChildren = children.filter {
            childrenByParentWithOneChildren[it] == null
        }
        put(parent, newChildren)
    }

    syntheticByParent.forEach { parent, synthetic ->
        val alreadyPreeasent = get(parent) ?: emptyList()

        put(parent, alreadyPreeasent + synthetic)
    }

    putAll(childrenBySynthetic)
}

mappingWithouthSynthStyle.values.flatten()

[e, bcd]

In [61]:
val transitions = mappingWithouthSynthStyle.flatMap { (parent, children) ->
    children.map { child ->
        Transition(
            start = parent,
            activity = (child as State.PrefixState).via,
            end = child
        )
    }
}

[Transition(start=c, activity=f, end=f), Transition(start=c, activity=eh, end=eh), Transition(start=root, activity=a, end=a), Transition(start=a, activity=c, end=c), Transition(start=a, activity=bdg, end=bdg), Transition(start=eh, activity=i, end=i), Transition(start=eh, activity=j, end=j)]

In [172]:
val syntheticByChainparts = buildMap {
    syntheticStateByChain.forEach { (chain, synth) ->
        chain.forEach { state ->
            put(state, synth)
        }
    }
}
syntheticByChainparts

{a=abc, b=abc, c=abc, d=de, e=de, b=bcde, c=bcde, d=bcde, e=bcde}

In [83]:
preStates
    .map { state ->
        if (state is State.PrefixState) {
            val from = state.from
            if (chains.flatten().contains(from)) {
                val newFrom = syntheticForPart[from]!!
                State.PrefixState(
                    from = newFrom,
                    via = state.via
                )
            } else state
        } else {
            state
        }
    }.map { (it as? State.PrefixState)?.from }

[eh, eh, null, root, c, eh, a, bdg]